# Auction house performance review

---

## The brief

> **Client:** Stockholms Auktionsverk Magasin 5
>
> **Your role:** Junior data analyst at a consulting firm
>
> **Context:** Stockholms Auktionsverk Magasin 5 is one of Sweden's largest auction houses. They've hired your firm to analyze their competitive position and identify growth opportunities.
>
> You have access to historical auction data from Auctionet covering multiple auction houses across Sweden.
>
> **Your task:** Deliver a data-driven competitive analysis. Your analysis should produce clear, well-formatted tables and include written insights interpreting your findings.

---

## Available data

Download the data files from Google Drive and place them in the same folder as this notebook:
- `lab_auctions.csv`
- `lab_bids.csv`

<details>
<summary><strong>Click here for full column reference</strong></summary>

### Auctions dataset (lab_auctions.csv)
| Column | Description |
|--------|-------------|
| `url` | Full URL to the auction item page |
| `id` | Unique internal identifier for the item (Auctionet ID) |
| `auction_id` | Unique identifier for the auction event this item belongs to |
| `currency` | Currency used for bidding (e.g., SEK, EUR) |
| `reserve_met` | Boolean indicating whether the reserve price has been met |
| `estimate` | Estimated price (lower bound) |
| `upper_estimate` | Estimated price (upper bound) |
| `next_bid_amount` | The minimum next bid amount required |
| `state` | Current state of the auction (e.g., active, ended) |
| `hammered` | Boolean, has the item been at a physical auction |
| `processed` | Boolean, has the auction been fully processed by the system? |
| `publicly_visible` | Boolean, is the auction publicly visible? |
| `ends_at` | Timestamp of when the auction ends |
| `ends_at_string` | Human-readable string for end time |
| `ends_at_string_short` | Short formatted version of end time |
| `ends_at_string_short_with_time` | Short formatted version including time |
| `license_weapon` | Boolean, item requires a weapons license |
| `online_auction` | Boolean, indicates if the auction is online |
| `hammer_auction` | Boolean, indicates if the auction is a physical hammer auction |
| `live_bidding_event_starts_at` | Start time for live bidding event |
| `live_bidding_event_approximate_ends_at` | Approximate end time for live event |
| `location` | Physical location of the item |
| `calendar_days_left` | Number of calendar days left before auction ends |
| `title` | Title of the item |
| `description` | Full description of the item |
| `condition` | Condition report |
| `house` | Auction house responsible for the item |
| `number_of_visits` | Number of times the item page has been visited |
| `main_category` | Main category of the item |
| `sub_category` | Sub-category of the item |
| `catalog_nr` | Catalog number for the item |
| `address` | Address where the item is located |
| `placement` | Item placement or lot number within the auction |
| `error` | Error status if any occurred in data retrieval |

### Bids dataset (lab_bids.csv)
| Column | Description |
|--------|-------------|
| `auction_id` | Foreign key linking to the auction item |
| `id` | Unique identifier for the bid |
| `bidder` | Identifier or anonymized ID of the bidder |
| `amount` | Amount of the bid |
| `reserve_met` | Boolean, indicates if this bid met the reserve |
| `first_to_meet_non_minimum_reserve_amount` | Boolean, indicates if this bid was the first to meet a non-minimum reserve |
| `auto` | Boolean, indicates if the bid was placed automatically (auto-bid) |
| `time_string_short` | Short formatted timestamp of bid placement |
| `time_string_long` | Long formatted timestamp of bid placement |
| `max_bid_time_string` | Long formatted timestamp of the bidder's max auto-bid submission |
</details>

## 1. Setup and data loading

Import the libraries you need and load the data files.

In [1]:
import pandas as pd
import numpy as np

### a. Import the data

In [2]:
auctions = pd.read_csv('lab_auctions.csv')
bids = pd.read_csv('lab_bids.csv')

In [3]:
auctions.head(5)

,url,id,auction_id,currency,reserve_met,estimate,upper_estimate,next_bid_amount,state,hammered,...,description,condition,house,number_of_visits,main_category,sub_category,catalog_nr,address,placement,error
0,https://auctionet.com/sv/1849226-ulrika-hydman...,1849226.0,3264671.0,SEK,True,1000.0,NaN,600.0,sold,False,...,"Höjd 26,5 cm.",Gott skick.,Hälsinglands Auktionsverk,361.0,Glas,Övrigt,NaN,NaN,NaN,NaN
1,https://auctionet.com/sv/1862268-notknappare-s...,1862268.0,3284950.0,SEK,True,420.0,NaN,489.0,sold,False,...,"Nötknäpparen, 18,5 cm hög.Tärningskastaren, 22...",Båda i begagnat skick.Nötknäpparen tråden någo...,The Auction Hub,312.0,NaN,NaN,NaN,NaN,NaN,NaN
2,https://auctionet.com/sv/1936203-lampetter-1-p...,1936203.0,3406868.0,SEK,True,600.0,NaN,610.0,sold,False,...,Mässing. Dekor av blommor och bladverk. En lju...,Ej funktionstestade. Slitage sladdar.,Gomér & Andersson Nyköping,237.0,Belysning & Lampor,Vägglampor,NaN,NaN,NaN,NaN
3,https://auctionet.com/sv/1664049-bordsur-brons...,1664049.0,2938688.0,SEK,True,2500.0,NaN,1600.0,sold,False,...,"Höjd 40 cm, bredd 20 cm.",Glas samt en visare saknas.,Halmstads Auktionskammare,287.0,Klockor & Ur,Bordsur,NaN,NaN,NaN,NaN
4,https://auctionet.com/sv/1744967-kikarsikte-ha...,1744967.0,3073932.0,SEK,True,600.0,NaN,350.0,sold,False,...,3-9x32.Längd ca 32 cm.,Ytslitage. Repor och märken.,Gomér & Andersson Nyköping,750.0,"Foto, Kameror & Optik",Övrigt,NaN,NaN,NaN,NaN


In [4]:
bids.head(5)

,Unnamed: 0,auction_id,id,bidder,amount,reserve_met,first_to_meet_non_minimum_reserve_amount,auto,time_string_short,time_string_long,max_bid_time_string
0,18,2949567,23740203,5,5600,True,NaN,False,"29 okt, 10:21",29 okt 2023 kl. 10:21:06 CET,"29 okt 2023, 10:21"
1,19,2949567,23740128,7,5100,True,NaN,True,"29 okt, 10:18",29 okt 2023 kl. 10:18:23 CET,"29 okt 2023, 10:16"
2,20,2949567,23740127,5,5000,True,NaN,False,"29 okt, 10:18",29 okt 2023 kl. 10:18:23 CET,"29 okt 2023, 10:18"
3,21,2949567,23740054,7,4300,True,NaN,False,"29 okt, 10:16",29 okt 2023 kl. 10:16:03 CET,"29 okt 2023, 10:16"
4,22,2949567,23697769,8,4100,True,NaN,False,"27 okt, 11:09",27 okt 2023 kl. 11:09:52 CEST,"27 okt 2023, 11:09"


## 2. Data exploration

Get familiar with the data. Check the shape, columns, data types, and look at some sample rows.

### a. Check info of both data

In [5]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10727 entries, 0 to 10726
Data columns (total 34 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   url                                     10727 non-null  object 
 1   id                                      10727 non-null  float64
 2   auction_id                              10727 non-null  float64
 3   currency                                10727 non-null  object 
 4   reserve_met                             10727 non-null  bool   
 5   estimate                                10727 non-null  float64
 6   upper_estimate                          9 non-null      float64
 7   next_bid_amount                         10727 non-null  float64
 8   state                                   10727 non-null  object 
 9   hammered                                10727 non-null  bool   
 10  processed                               10727 non-null  bo

In [6]:
bids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490007 entries, 0 to 2490006
Data columns (total 11 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Unnamed: 0                                int64  
 1   auction_id                                int64  
 2   id                                        int64  
 3   bidder                                    int64  
 4   amount                                    int64  
 5   reserve_met                               bool   
 6   first_to_meet_non_minimum_reserve_amount  float64
 7   auto                                      bool   
 8   time_string_short                         object 
 9   time_string_long                          object 
 10  max_bid_time_string                       object 
dtypes: bool(2), float64(1), int64(5), object(3)
memory usage: 175.7+ MB


#### **Findings** :  The auction id and id also bidder of both data are integer and float we need to change it to object type. Also we need to change the amount data type to float in bids data frame to make it easier to analyze with the estimate column in auctions dataframe. I also will drop unnamed column that supposed to be not in the dataframe.

### b.Changing data type of auction_id and id

In [7]:
auctions['id'] = auctions['id'].astype(object) 
auctions['auction_id'] = auctions['id'].astype(object) 
bids['id'] = bids['id'].astype(object) 
bids['auction_id'] = bids['auction_id'].astype(object) 
bids['bidder'] = bids['bidder'].astype(object) 
bids['amount'] = bids['amount'].astype('float') 

In [8]:
bids = bids.drop('Unnamed: 0', axis=1)

In [9]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10727 entries, 0 to 10726
Data columns (total 34 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   url                                     10727 non-null  object 
 1   id                                      10727 non-null  object 
 2   auction_id                              10727 non-null  object 
 3   currency                                10727 non-null  object 
 4   reserve_met                             10727 non-null  bool   
 5   estimate                                10727 non-null  float64
 6   upper_estimate                          9 non-null      float64
 7   next_bid_amount                         10727 non-null  float64
 8   state                                   10727 non-null  object 
 9   hammered                                10727 non-null  bool   
 10  processed                               10727 non-null  bo

In [10]:
bids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490007 entries, 0 to 2490006
Data columns (total 10 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   auction_id                                object 
 1   id                                        object 
 2   bidder                                    object 
 3   amount                                    float64
 4   reserve_met                               bool   
 5   first_to_meet_non_minimum_reserve_amount  float64
 6   auto                                      bool   
 7   time_string_short                         object 
 8   time_string_long                          object 
 9   max_bid_time_string                       object 
dtypes: bool(2), float64(2), object(6)
memory usage: 156.7+ MB


### c. check any duplicate

In [11]:
auctions.duplicated().sum()

np.int64(0)

In [12]:
bids.duplicated().sum()

np.int64(0)

#### **Findings**: No duplicate for both data frame

### d. Check Null and nan value for estimate, upper estimate, and amount column

In [13]:
auctions['estimate'].isnull().sum()

np.int64(0)

In [14]:
mask = auctions['upper_estimate'].isnull().sum()
mask

np.int64(10718)

In [15]:
percent_null_upper = mask/len(auctions['upper_estimate'])
percent_null_upper.round(3)

np.float64(0.999)

In [16]:
bids['amount'].isnull().sum()

np.int64(0)

#### **Findings** = As we see from the result, estimate and amount column doesnt have any null value, but upper estimate has 99% of null value. So i decided to drop or not use the column.

### e. Check the outlier

In [17]:
auctions["estimate"].nlargest(5)

2277     160000.0
10461    152953.0
8465      98518.0
3613      80000.0
5997      61461.0
Name: estimate, dtype: float64

In [18]:
auctions[auctions["estimate"] > 160000]

,url,id,auction_id,currency,reserve_met,estimate,upper_estimate,next_bid_amount,state,hammered,...,description,condition,house,number_of_visits,main_category,sub_category,catalog_nr,address,placement,error


In [19]:
bids["amount"].nlargest(5)

1700742    1263163.0
1700743    1213168.0
1700744    1213160.0
1700745    1163160.0
1700746    1113160.0
Name: amount, dtype: float64

In [20]:
bids[bids["amount"] > 1263163]

,auction_id,id,bidder,amount,reserve_met,first_to_meet_non_minimum_reserve_amount,auto,time_string_short,time_string_long,max_bid_time_string


In [21]:
auctions.describe()

,estimate,upper_estimate,next_bid_amount,ends_at,live_bidding_event_starts_at,live_bidding_event_approximate_ends_at,calendar_days_left,number_of_visits,catalog_nr,error
count,10727.000000,9.000000,10727.000000,1.072700e+04,0.0,0.0,10727.0,10727.000000,0.0,0.0
mean,2238.070476,13136.444444,2456.593269,1.617786e+09,NaN,NaN,0.0,410.717162,NaN,NaN
std,4355.159120,15122.941026,11439.383273,4.551060e+07,NaN,NaN,0.0,254.854315,NaN,NaN
min,300.000000,4637.000000,250.000000,1.348335e+09,NaN,NaN,0.0,1.000000,NaN,NaN
25%,600.000000,6955.000000,450.000000,1.611513e+09,NaN,NaN,0.0,262.000000,NaN,NaN
50%,1000.000000,6955.000000,900.000000,1.634485e+09,NaN,NaN,0.0,405.000000,NaN,NaN
75%,2000.000000,9273.000000,2400.000000,1.644265e+09,NaN,NaN,0.0,580.000000,NaN,NaN
max,160000.000000,52157.000000,760011.000000,1.701276e+09,NaN,NaN,0.0,999.000000,NaN,NaN


In [22]:
bids.describe()

,amount,first_to_meet_non_minimum_reserve_amount
count,2.490007e+06,139709.000000
mean,2.192874e+03,2425.478387
std,6.583470e+03,4890.619319
min,2.000000e+02,250.000000
25%,5.000000e+02,600.000000
50%,9.070000e+02,1100.000000
75%,2.000000e+03,2500.000000
max,1.263163e+06,463612.000000


#### **Findings** = There no outlier in both dataframe

### f. Checking if currency has EURO 

In [113]:
auctions.currency.value_counts()

currency
SEK    10727
Name: count, dtype: int64

#### **Findings** = There no Euro in currency

### g. Find the highest bids before join the table

In [23]:
max_bids = bids.groupby('auction_id', as_index=False)['amount'].max()
max_bids['bidder'] = bids['bidder']
max_bids

,auction_id,amount,bidder
0,10047,600.0,5
1,10178,5110.0,7
2,10342,1200.0,5
3,10360,1200.0,7
4,10463,300.0,8
...,...,...,...
279837,3876212,4604.0,2
279838,3876478,1062.0,4
279839,3876495,1000.0,2
279840,3876516,1391.0,4


---

## 3. Market position

Help the client understand where they stand in the market.

### What is our overall market share?

Calculate the percentage of all auctions that belong to Stockholms Auktionsverk Magasin 5.

In [24]:
auction2 = pd.merge(auctions, max_bids, on='auction_id', how='left')
auction2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10727 entries, 0 to 10726
Data columns (total 36 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   url                                     10727 non-null  object 
 1   id                                      10727 non-null  object 
 2   auction_id                              10727 non-null  object 
 3   currency                                10727 non-null  object 
 4   reserve_met                             10727 non-null  bool   
 5   estimate                                10727 non-null  float64
 6   upper_estimate                          9 non-null      float64
 7   next_bid_amount                         10727 non-null  float64
 8   state                                   10727 non-null  object 
 9   hammered                                10727 non-null  bool   
 10  processed                               10727 non-null  bo

In [134]:
auction2.house.value_counts(normalize= True).round(2).reset_index().head(5)

,house,proportion
0,Gomér & Andersson Linköping,0.07
1,Stockholms Auktionsverk Magasin 5,0.06
2,Växjö Auktionskammare,0.05
3,Stadsauktion Sundsvall,0.05
4,Auktionshuset Kolonn,0.05


**Your findings:**

According to my findings, Stockholms Auctionsverk Magasin 5 has 6% of overall market share.

### How does our market share vary by category?

In which categories do we have the highest share? The lowest?

In [132]:
auction2[auction2['house'] == 'Stockholms Auktionsverk Magasin 5']['main_category'].value_counts(normalize=True).round(3).reset_index()

,main_category,proportion
0,Vapen & Militaria,0.191
1,Belysning & Lampor,0.190
2,Smycken & Ädelstenar,0.130
3,Klockor & Ur,0.113
4,"Böcker, Kartor & Handskrifter",0.102
5,Allmoge,0.086
6,Glas,0.064
7,Möbler,0.059
8,Silver & Metall,0.025
9,"Mynt, Medaljer & Frimärken",0.019


In [131]:
auction2[auction2['house'] == 'Stockholms Auktionsverk Magasin 5'].groupby('main_category').size().sort_values(ascending=False).reset_index()

,main_category,0
0,Vapen & Militaria,113
1,Belysning & Lampor,112
2,Smycken & Ädelstenar,77
3,Klockor & Ur,67
4,"Böcker, Kartor & Handskrifter",60
5,Allmoge,51
6,Glas,38
7,Möbler,35
8,Silver & Metall,15
9,"Mynt, Medaljer & Frimärken",11


In [130]:
auction2[auction2['house'] == 'Stockholms Auktionsverk Magasin 5']['sub_category'].value_counts(normalize=True).round(3).reset_index()

,sub_category,proportion
0,Övrigt,0.124
1,Ljuskronor,0.100
2,Blankvapen,0.076
3,Vägglampor,0.063
4,Skjutvapen,0.061
5,Kartor,0.056
6,Matsalsmöbler,0.052
7,Fickur & Stoppur,0.046
8,Preciosa & Övrigt,0.044
9,Ädelstenar,0.044


**Your findings:**

According to my findings, for main category, Stockholms Auktionsverk Magasin 5 is dominating on Vapen & Militaria (19%) and Belysning & Lampor(19%). For Sub category, Övrigt(12%) and  Ljuskronor(10%)

### Who are our main competitors?

Identify the largest auction houses and compare them to our client.

In [129]:
auction2[auction2['main_category']== 'Vapen & Militaria']['house'].value_counts(normalize=True).round(3).reset_index().head(5)

,house,proportion
0,Stockholms Auktionsverk Magasin 5,0.110
1,Stadsauktion Sundsvall,0.089
2,Garpenhus Auktioner,0.068
3,Halmstads Auktionskammare,0.050
4,RA Auktionsverket Norrköping,0.049


In [128]:
auction2[auction2['main_category']== 'Belysning & Lampor']['house'].value_counts(normalize=True).round(3).reset_index().head(5)

,house,proportion
0,Stockholms Auktionsverk Magasin 5,0.115
1,Stadsauktion Sundsvall,0.075
2,Stockholms Auktionsverk Sickla,0.062
3,Gomér & Andersson Norrköping,0.057
4,Helsingborgs Auktionskammare,0.057


In [127]:
auction2[auction2['sub_category'] == 'Övrigt']['house'].value_counts(normalize=True).round(3).reset_index().head(5)

,house,proportion
0,Växjö Auktionskammare,0.094
1,Auktionshuset Kolonn,0.061
2,Gomér & Andersson Linköping,0.059
3,Kalmar Auktionsverk,0.058
4,Stockholms Auktionsverk Magasin 5,0.053


In [126]:
auction2[auction2['sub_category'] == 'Ljuskronor']['house'].value_counts(normalize=True).round(3).reset_index().head(5)

,house,proportion
0,Stockholms Auktionsverk Magasin 5,0.260
1,Stockholms Auktionsverk Sickla,0.097
2,Helsingborgs Auktionskammare,0.066
3,Gomér & Andersson Linköping,0.053
4,Stadsauktion Sundsvall,0.053


**Your findings:**

According to my findings, Stadsauktion Sundsvall stands out as the most consistent challenger, maintaining a top-five presence across three different categories. Regional competitors like Gomér & Andersson and Helsingborgs Auktionskammare also quite significant for mid-tier volume, particularly within the lighting segments. Meanwhile, houses like Garpenhus Auktioner is showing strength in "Vapen & Militaria" while remaining absent from other lists.

---

## 4. Performance analysis

Dig deeper into how well the client performs compared to competitors.

### How accurate are our price estimates compared to competitors?

Compare the ratio of final sale price to estimate across auction houses. A ratio > 1 means items sell above estimate.

*You'll need to join the bids data to find the final (highest) bid for each auction.*

### a. calculating abs diff and mean of amount and estimate 

In [33]:
auction2['diff_price_with_estimate'] = auction2['amount'] - auction2['estimate']
auction2['abs_diff'] = abs(auction2['diff_price_with_estimate'])
auction2['abs_diff']

auction2['percentage_diff']  = auction2['abs_diff'] / auction2['estimate']
auction2['percentage_diff'].mean().round(2)

np.float64(0.55)

**Your findings:**

According to my findings the ratio is 0.55. It likely means the typical item sold for roughly 55% of its estimated value.

### Which categories attract the most bidding activity?

Calculate the average number of bids per auction, broken down by category.

### a. Based on bidder column

In [125]:
auction2.groupby('main_category')['bidder'].size().sort_values(ascending=False).reset_index()

,main_category,bidder
0,Smycken & Ädelstenar,1766
1,Klockor & Ur,1360
2,Allmoge,1088
3,Vapen & Militaria,1028
4,Belysning & Lampor,972
5,Glas,839
6,Möbler,591
7,Licensvapen,525
8,"Foto, Kameror & Optik",493
9,"Mynt, Medaljer & Frimärken",418


### b. Based on amount column

In [124]:
auction2.groupby('main_category')['amount'].sum().sort_values(ascending=False).reset_index()

,main_category,amount
0,Smycken & Ädelstenar,1802815.0
1,Vapen & Militaria,1595757.0
2,Licensvapen,1566607.0
3,Klockor & Ur,1269398.0
4,Möbler,1079598.0
5,"Böcker, Kartor & Handskrifter",1038664.0
6,Belysning & Lampor,779580.0
7,Allmoge,756984.0
8,"Mynt, Medaljer & Frimärken",752728.0
9,Glas,452789.0


**Your findings:**

Smycken & Ädelstenar is the top category for bidding activity with 1.766 total bidder with total amount around SEK 1.8 million, then followed by Klockor & Ur (1.360 total bidder) and Allmoge(1.088 total bidder). In term of amount of money Vapen & Militaria is number 2 with SEK 1.59 million and Licensvapen is number 3 with SEK 1.56 million

### How well do our listings attract visitors?

Compare average visitor numbers across the top auction houses.

In [135]:
pd.set_option('display.max_rows', None)
visitors = auction2.groupby('house')['number_of_visits'].mean().sort_values(ascending=False).reset_index()
visitors['rank'] = visitors['number_of_visits'].rank(ascending=False)
visitors.head(20)

,house,number_of_visits,rank
0,Auction House Old Eminence,763.000000,1.0
1,Auktionshaus Dannenberg,722.000000,2.0
2,Frufällans Auktionshus,669.000000,3.0
3,Art and Design Auctions,603.700000,4.0
4,Stockholms Auktionsverk Norrköping,512.333333,5.0
5,HannoVerum,496.354839,6.0
6,Auktionisten i Sörmland,487.250000,7.0
7,Stockholms Auktionsverk Malmö,483.270588,8.0
8,Herrljunga Auktionsverk,477.136364,9.0
9,Stockholms Auktionsverk Düsseldorf/Neuss,475.754098,10.0


**Your findings:**

According to my findings, the visitor for our client is moderate around 456 customer per visit, and ranked 15 from all of the houses.

---

## 5. Strategic insights

Provide strategic recommendations based on your analysis.

### What growth opportunities exist for our client?

Find categories where the market is large but our client has low presence.

### a. Growth Opportunities based on Main Category

In [74]:
# Count total main category
total_category = auction2['main_category'].value_counts().reset_index()
total_category

,main_category,count
0,Smycken & Ädelstenar,1766
1,Klockor & Ur,1360
2,Allmoge,1088
3,Vapen & Militaria,1028
4,Belysning & Lampor,972
5,Glas,839
6,Möbler,591
7,Licensvapen,525
8,"Foto, Kameror & Optik",493
9,"Mynt, Medaljer & Frimärken",418


In [76]:
# Count total category for our client only
total_category_client = auction2[auction2['house'] == 'Stockholms Auktionsverk Magasin 5']['main_category'].value_counts().reset_index()
total_category_client

,main_category,count
0,Vapen & Militaria,113
1,Belysning & Lampor,112
2,Smycken & Ädelstenar,77
3,Klockor & Ur,67
4,"Böcker, Kartor & Handskrifter",60
5,Allmoge,51
6,Glas,38
7,Möbler,35
8,Silver & Metall,15
9,"Mynt, Medaljer & Frimärken",11


In [67]:
# Merge the above data with left join
opportunity = pd.merge(total_category, total_category_client, on='main_category', how='left')
opportunity

,main_category,count_x,count_y
0,Smycken & Ädelstenar,1766,77.0
1,Klockor & Ur,1360,67.0
2,Allmoge,1088,51.0
3,Vapen & Militaria,1028,113.0
4,Belysning & Lampor,972,112.0
5,Glas,839,38.0
6,Möbler,591,35.0
7,Licensvapen,525,NaN
8,"Foto, Kameror & Optik",493,3.0
9,"Mynt, Medaljer & Frimärken",418,11.0


In [72]:
# fill nan with 0
opportunity['count_y'] = opportunity['count_y'].fillna(0)
#count market share of each main category
opportunity['market_share'] = (opportunity['count_y'] / opportunity['count_x']).round(2)
opportunity.sort_values(ascending=False, by='count_x')

,main_category,count_x,count_y,market_share
0,Smycken & Ädelstenar,1766,77.0,0.04
1,Klockor & Ur,1360,67.0,0.05
2,Allmoge,1088,51.0,0.05
3,Vapen & Militaria,1028,113.0,0.11
4,Belysning & Lampor,972,112.0,0.12
5,Glas,839,38.0,0.05
6,Möbler,591,35.0,0.06
7,Licensvapen,525,0.0,0.00
8,"Foto, Kameror & Optik",493,3.0,0.01
9,"Mynt, Medaljer & Frimärken",418,11.0,0.03


### b. Growth Opportunities based on Sub Category

In [81]:
# Count total sub category
total_subcategory = auction2['sub_category'].value_counts().reset_index()
total_subcategory.head(5)

,sub_category,count
0,Övrigt,1382
1,Skjutvapen,532
2,Matsalsmöbler,507
3,Fickur & Stoppur,463
4,Preciosa & Övrigt,394


In [82]:
# Count total sub category for our client only
total_subcategory_client = auction2[auction2['house'] == 'Stockholms Auktionsverk Magasin 5']['sub_category'].value_counts().reset_index()
total_subcategory_client.head(5)

,sub_category,count
0,Övrigt,73
1,Ljuskronor,59
2,Blankvapen,45
3,Vägglampor,37
4,Skjutvapen,36


In [83]:
# Merge the above data with left join
sub_opportunity = pd.merge(total_subcategory, total_subcategory_client, on='sub_category', how='left')
sub_opportunity.head(5)

,sub_category,count_x,count_y
0,Övrigt,1382,73.0
1,Skjutvapen,532,36.0
2,Matsalsmöbler,507,31.0
3,Fickur & Stoppur,463,27.0
4,Preciosa & Övrigt,394,26.0


In [80]:
# fill nan with 0
sub_opportunity['count_y'] = sub_opportunity['count_y'].fillna(0)
#count market share of each main category
sub_opportunity['market_share'] = (sub_opportunity['count_y'] / opportunity['count_x']).round(2)
sub_opportunity.sort_values(ascending=False, by='count_x').head(5)

,sub_category,count_x,count_y,market_share
0,Övrigt,1382,73.0,0.04
1,Skjutvapen,532,36.0,0.03
2,Matsalsmöbler,507,31.0,0.03
3,Fickur & Stoppur,463,27.0,0.03
4,Preciosa & Övrigt,394,26.0,0.03


**Your findings:**

According to my findings, for main category: Smycken & Ädelstenar, Klockor & Ur and Allmoge has big opportunities to explore and fill for our client. and for sub category, Skjutvapen, Matsalsmöbler and Fickur & Stoppur has great opportunity to be explored by our client.

### Which competitor should we study for best practices?

Based on your analysis, which competitor performs best overall? What could our client learn from them?

In [122]:
# calculating each sales ratio for our competitior and compare with our client's ratio
competitor = auction2.groupby('house').agg({'bidder': 'count','number_of_visits': 'mean','amount': 'mean','estimate': 'mean'})
competitor['ratio_sales'] = (competitor['amount'] /  competitor['estimate']).round(2)
competitor.sort_values(ascending=False, by='ratio_sales').reset_index().head(10)

,house,bidder,number_of_visits,amount,estimate,ratio_sales
0,Södermanlands Auktionsverk,161,420.235110,5658.173913,1633.542320,3.46
1,Göteborgs Auktionsverk,100,455.026786,4738.580000,3063.392857,1.55
2,Lysekils Auktionsbyrå,24,454.860000,4109.125000,2856.000000,1.44
3,Art and Design Auctions,4,603.700000,4969.750000,3547.100000,1.40
4,Crafoord Auktioner Lund,222,443.693548,3092.306306,2218.087558,1.39
5,Stockholms Auktionsverk Helsingborg,52,440.902913,3777.384615,2783.495146,1.36
6,Walter Borg,174,298.474320,6779.235632,5024.622356,1.35
7,Stockholms Auktionsverk Magasin 5,324,455.618968,3581.737654,2835.773710,1.26
8,Stockholms Auktionsverk Göteborg,29,474.338028,3338.758621,2671.830986,1.25
9,Stockholms Auktionsverk Helsinki,22,407.490196,5629.227273,4636.901961,1.21


**Your findings:**

As you can see that our client rank number 8 in terms of ratio sales, what we need to study from our competitor is that how they can have such a high sales ration like Södermanlands Auktionsverk	that has 3.46 sales ratio and Göteborgs Auktionsverk that has 1.55 sales ratio.

### Your own insight

Find one additional insight from the data that would be valuable for the client.

### I will explore what kind of product in main category and sub category of Södermanlands Auktionsverk so we can have the same products for our client

#### a. Main category for Competitor

In [121]:
total_category_compe = auction2[auction2['house'] == 'Södermanlands Auktionsverk']['main_category'].value_counts()
opportunity_compe = pd.merge(total_category, total_category_compe, on='main_category', how='left')

# fill nan with 0
opportunity_compe['count_y'] = opportunity_compe['count_y'].fillna(0)
#count market share of each main category
opportunity_compe['market_share'] = (opportunity_compe['count_y'] / opportunity_compe['count_x']).round(2)
opportunity_compe.sort_values(ascending=False, by='market_share').reset_index()


,index,main_category,count_x,count_y,market_share
0,6,Möbler,591,39.0,0.07
1,2,Allmoge,1088,52.0,0.05
2,8,"Foto, Kameror & Optik",493,27.0,0.05
3,12,Keramik & Porslin,57,3.0,0.05
4,13,Leksaker,43,2.0,0.05
5,1,Klockor & Ur,1360,53.0,0.04
6,10,Silver & Metall,226,9.0,0.04
7,0,Smycken & Ädelstenar,1766,45.0,0.03
8,3,Vapen & Militaria,1028,35.0,0.03
9,4,Belysning & Lampor,972,25.0,0.03


#### b. Sub Category for Competitor

In [120]:
total_subcategory_compe = auction2[auction2['house'] == 'Södermanlands Auktionsverk']['sub_category'].value_counts()
opportunitysub_compe = pd.merge(total_subcategory, total_subcategory_compe, on='sub_category', how='left')

# fill nan with 0
opportunitysub_compe['count_y'] = opportunitysub_compe['count_y'].fillna(0)
#count market share of each main category
opportunitysub_compe['market_share'] = (opportunitysub_compe['count_y'] / opportunitysub_compe['count_x']).round(2)
opportunitysub_compe.sort_values(ascending=False, by='market_share').reset_index().head(5)


,index,sub_category,count_x,count_y,market_share
0,31,Trädgårdsmöbler,84,7.0,0.08
1,7,Väggur,333,27.0,0.08
2,11,Möbler,294,23.0,0.08
3,13,Ädelstenar,277,19.0,0.07
4,23,Golvur,149,9.0,0.06


#### c. Sales ratio of Competitor based on Main Category

In [119]:
compe_soder = auction2[auction2['house'] == 'Södermanlands Auktionsverk'].groupby('main_category').agg({'amount': 'mean','estimate': 'mean'})
compe_soder['ratio_sales'] = (compe_soder['amount'] /  compe_soder['estimate']).round(2)
compe_soder.sort_values(ascending=False, by='ratio_sales').reset_index()

,main_category,amount,estimate,ratio_sales
0,"Böcker, Kartor & Handskrifter",150272.200000,3983.333333,37.73
1,Belysning & Lampor,934.500000,856.000000,1.09
2,Klockor & Ur,753.916667,784.905660,0.96
3,Glas,810.000000,860.000000,0.94
4,Keramik & Porslin,385.000000,466.666667,0.82
5,"Foto, Kameror & Optik",634.545455,803.703704,0.79
6,Leksaker,400.000000,600.000000,0.67
7,Möbler,1839.304348,2735.897436,0.67
8,Silver & Metall,521.500000,877.777778,0.59
9,Allmoge,662.681818,1346.153846,0.49


#### d. Sales ratio of Competitor based on sub Category

In [118]:
compe_soder = auction2[auction2['house'] == 'Södermanlands Auktionsverk'].groupby('sub_category').agg({'amount': 'mean','estimate': 'mean'})
compe_soder['ratio_sales'] = (compe_soder['amount'] /  compe_soder['estimate']).round(2)
compe_soder.sort_values(ascending=False, by='ratio_sales').reset_index().head(5)

,sub_category,amount,estimate,ratio_sales
0,Kartor,375155.500000,7000.000000,53.59
1,Fickur & Stoppur,1825.666667,812.500000,2.25
2,Vägglampor,1401.666667,685.714286,2.04
3,Kikare & Vapenoptik,1333.333333,975.000000,1.37
4,Smycken,753.000000,637.500000,1.18


**Your findings:**

As we know that, Stockholms Auktionsverk Magasin 5 is dominating on Vapen & Militariaand Belysning & Lampor. For Sub category, Övrigt and Ljuskronor. So for our client, to imporve their business, based on exploring our competition, Stockholms Auktionsverk Magasin 5 need to increase their market share in Möbler and Allmoge main category. And for sub category they need to sell Trädgårdsmöbler, Väggur and Möbler.

but in terms of sales ratio, they need to sell Böcker, Kartor & Handskrifter to earn more money in the main category. And for sub category are Kartor and Fickur & Stoppur.

---

## Executive summary

Write a brief summary (3-5 bullet points) with your key findings and recommendations for Stockholms Auktionsverk Magasin 5.

---

**Your recommendations:**

1. Stockholms Auktionsverk Magasin 5 has 6% overall market share, and dominating in Vapen & Militaria and Belysning & Lampor with 19% share in both.
2. Stockholms Auktionsverk Magasin 5 sales ratio is currently ranked #8. When looking at competitors like Södermanlands (who have a massive 3.46 ratio), it’s clear we have room to improve how efficiently we convert listings into sales.
3. To get Stockholms Auktionsverk Magasin 5 visitor numbers up (currently avg. 456), I recommend we aggressively explore Smycken and Klockor. The data shows this is where the highest volume of bidders are (over 1,700), and we are missing out on that traffic.
4. Stockholms Auktionsverk Magasin 5 needs to increase market share in the Möblerand Allmoge categories. Specifically, the sub-categories of Trädgårdsmöbler and Väggur are key areas where we can quickly gain market share by increasing our volume of listings.
5. To boost Stockholms Auktionsverk Magasin 5 stats quickly, the numbers suggest stocking up on Böcker & Kartor and Fickur & Stoppur. These categories look like low-hanging fruit to improve our sales ratio compared to just chasing volume.
   